# QnA Matching Tutorial (Phrase Learning, Model Training & Evaluation)

## Overview

First part of this notebook shows how to pre-process the text data, learn the most salient phrases present in a large collection of documents and save cleaned text data in the Azure Blob Storage. These phrases can be treated as single compound word units in down-stream processes such as discriminative training. To learn the phrases, we have implemented the basic framework that combines key phrase learning and latent topic modeling as described in the paper entitled ["Modeling Multiword Phrases with Constrained Phrases Tree for Improved Topic Modeling of Conversational Speech"](http://people.csail.mit.edu/hazen/publications/Hazen-SLT-2012.pdf) which was originally presented in the 2012 IEEE Workshop on Spoken Language Technology. Although the paper examines the use of the technology for analyzing human-to-human conversations, the techniques are quite general and can be applied to a wide range of natural language data including news stories, legal documents, research publications, social media forum discussions, customer feedback forms, product reviews, and many more.

The remaining part of the notebooks demonstrate various approaches for feature extraction, model training and evaluation.

Note: This notebook series are built under Python 3.5 and NLTK 3.2.2.

## Import Required Python Modules

In this notebook, we use several open-source Python packages that need to be installed in a local machine or an Azure Notebook Server. An upgrade is requested if a previous version of a package has been installed in the past.

We make use of the NLTK sentence tokenization capability which takes a long string of text and splits it into sentence units. The tokenizer requires the installation of the 'punkt' tokenizer models. After importing nltk, the nltk.download() function can be used to download specific packages such as 'punkt'.

In [1]:
# uncomment the below code to install/upgrade the requested Python packages.
# !pip install --upgrade --no-deps smart_open azure pandas nltk

In [2]:
import pandas as pd
import numpy as np
import re, sys, os, gc, requests, time, math, nltk, glob, os.path, ftplib, json, base64, datetime, warnings
from collections import (namedtuple, Counter)
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from azure.storage import CloudStorageAccount
warnings.filterwarnings("ignore")

In [3]:
EMPTY = ''
SPACE = ' '
nltk.download("punkt")
NLTK_PUNKT_EN = 'tokenizers/punkt/english.pickle'
SENTENCE_BREAKER = nltk.data.load(NLTK_PUNKT_EN)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read trainQ and testQ into DataFrames

As we have prepared the _trainQ_ and _testQ_ from the previous notebook, we retrieve the datasets here for the further process.
_trainQ_ contains 5,153 training examples and _testQ_ contains 1,735 test examples. Also, there are 103 unique answer classes in both datasets.

In [4]:
# functions to load .txt file into a Python dictionary. 
def load_from_url(fileURL):
    response = requests.get(fileURL, stream=True)
    return response.text.split('\n')

def LoadListAsHash(fileURL):
    listHash = {}
    wordsList = load_from_url(fileURL)
    # Read in lines one by one stripping away extra spaces, 
    # leading spaces, and trailing spaces and inserting each
    # cleaned up line into a hash table
    re1 = re.compile(' +')
    re2 = re.compile('^ +| +$')
    for stringIn in wordsList:
        term = re2.sub("",re1.sub(" ",stringIn.strip('\n')))
        if term != '':
            listHash[term] = 1
    return listHash

In [5]:
trainQ_url = 'https://mezsa.blob.core.windows.net/stackoverflownew/trainQ_tutorial.tsv'
testQ_url = 'https://mezsa.blob.core.windows.net/stackoverflownew/testQ_tutorial.tsv'
# answersC_url = 'https://mezsa.blob.core.windows.net/stackoverflownew/answersC_tutorial.tsv'
function_words_url = 'https://mezsa.blob.core.windows.net/stackoverflow/function_words.txt'

# load datasets.
trainQ = pd.read_csv(trainQ_url, sep='\t', index_col='Id', encoding='latin1')
testQ = pd.read_csv(testQ_url, sep='\t', index_col='Id', encoding='latin1')
# answersC = pd.read_csv(answersC_url, sep='\t', index_col='Id', encoding='latin1')
# Load the list of non-content bearing function words
functionwordHash = LoadListAsHash(function_words_url)

## Phrase Learning

### Process Text Data

The CleanAndSplitText function below takes as input a list where each row element is a single cohesive long string of text, i.e. a "document". The function first splits each string by various forms of punctuation into chunks of text that are likely sentences, phrases or sub-phrases. The splitting is designed to prohibit the phrase learning process from using cross-sentence or cross-phrase word strings when learning phrases.

The function creates a table where each row represents a chunk of text from the original documents. The DocIndex coulmn indicates the original row index from associated document in the input from which the chunk of text originated. The TextLine column contains the original text excluding the punctuation marks and HTML markup that have been during the cleaning process. The TextLineLower column contains a fully lower-cased version of the text in the TextLine column.


In [6]:
def CleanAndSplitText(frame):

    textDataOut = [] 

    # This regular expression is for punctuation that we wish to clean out
    # We also will split sentences into smaller phrase like units using this expression
    rePhraseBreaks = re.compile("[\"\!\?\)\]\}\,\:\;\*\-]*\s+\([0-9]+\)\s+[\(\[\{\"\*\-]*"                         
                                "|[\"\!\?\)\]\}\,\:\;\*\-]+\s+[\(\[\{\"\*\-]*" 
                                "|\.\.+"       # ..
                                "|\s*\-\-+\s*" # --
                                "|\s+\-\s+"    # -  
                                "|\:\:+"       # ::
                                "|\s+[\/\(\[\{\"\-\*]+\s*"  
                                "|[\,!\?\"\)\(\]\[\}\{\:\;\*](?=[a-zA-Z])"
                                "|[\"\!\?\)\]\}\,\:\;]+[\.]*$"
                             )
    
    # Regex for underbars
    regexUnderbar = re.compile('_|_+')
    
    # Regex for space
    regexSpace = re.compile(' +')
 
    # Regex for sentence final period
    regexPeriod = re.compile("\.$")
    
    # Regex for parentheses
    regexParentheses = re.compile("\(\$?")
    
    # Regex for equal sign
    regexEqual = re.compile("=")

    # Iterate through each document and do:
    #    (1) Split documents into sections based on section headers and remove section headers
    #    (2) Split the sections into sentences using NLTK sentence tokenizer
    #    (3) Further split sentences into phrasal units based on punctuation and remove punctuation
    #    (4) Remove sentence final periods when not part of a abbreviation 

    for i in range(0,len(frame)):
        
        # Extract one document from frame
        docID = frame.index.values[i]
        docText = frame['Text'].iloc[i] 

        # Set counter for output line count for this document
        lineIndex=0

        sentences = SENTENCE_BREAKER.tokenize(docText)
        
        for sentence in sentences:

            # Split each sentence into phrase level chunks based on punctuation
            textSegs = rePhraseBreaks.split(sentence)
            numSegs = len(textSegs)

            for j in range(0,numSegs):
                if len(textSegs[j])>0:
                    # Convert underbars to spaces 
                    # Underbars are reserved for building the compound word phrases                   
                    textSegs[j] = regexUnderbar.sub(" ",textSegs[j])
                    
                    # Split out the words so we can specially handle the last word
                    words = regexSpace.split(textSegs[j])
                    
                    # Remove parentheses and equal signs
                    words = [regexEqual.sub("", regexParentheses.sub("", w)) for w in words]
                    
                    phraseOut = ""
                    last = len(words) -1
                    for i in range(0, last):
                        phraseOut += words[i] + " "
                    # If the last word ends in a period then remove the period
                    lastWord = regexPeriod.sub("", words[last])
                    # If the last word is an abbreviation like "U.S."
                    # then add the word final perios back on
                    if "\." in lastWord:
                        lastWord += "."
                    phraseOut += lastWord    

                    textDataOut.append([docID,lineIndex,phraseOut, phraseOut.lower()])
                    lineIndex += 1
                        
    # Convert to pandas frame 
    frameOut = pd.DataFrame(textDataOut, columns=['DocID','DocLine','CleanedText', 'LowercaseText'])                      
    
    return frameOut

In [7]:
CleanedTrainQ = CleanAndSplitText(trainQ)
CleanedTestQ = CleanAndSplitText(testQ)

In [8]:
CleanedTrainQ.head(2)

,DocID,DocLine,CleanedText,LowercaseText
0,69913,0,why don't self-closing script tags work,why don't self-closing script tags work
1,69913,1,what is the reason browsers do not correctly r...,what is the reason browsers do not correctly r...


### Compute N-gram Statistics for Phrase Learning 

In [9]:
# This is Step 1 for each iteration of phrase learning
# We count the number of occurances of all 2-gram, 3-ngram, and 4-gram
# word sequences 
def ComputeNgramStats(textData,functionwordHash,blacklistHash):
    
    # Create an array to store the total count of all ngrams up to 4-grams
    # Array element 0 is unused, element 1 is unigrams, element 2 is bigrams, etc.
    ngramCounts = [0]*5;
       
    # Create a list of structures to tabulate ngram count statistics
    # Array element 0 is the array of total ngram counts,
    # Array element 1 is a hash table of individual unigram counts
    # Array element 2 is a hash table of individual bigram counts
    # Array element 3 is a hash table of individual trigram counts
    # Array element 4 is a hash table of individual 4-gram counts
    ngramStats = [ngramCounts, {}, {}, {}, {}]
          
    # Create a regular expression for assessing validity of words
    # for phrase modeling. The expression says words in phrases
    # must either:
    # (1) contain an alphabetic character, or 
    # (2) be the single charcater '&', or
    # (3) be a one or two digit number
    reWordIsValid = re.compile('[A-Za-z]|^&$|^\d\d?$')
    
    # Go through the text data line by line collecting count statistics
    # for all valid n-grams that could appear in a potential phrase
    numLines = len(textData)
    for i in range(0, numLines):

        # Split the text line into an array of words
        wordArray = textData[i].split()
        numWords = len(wordArray)
#         if numWords == 0:
#             print(textData[i])
        
        # Create an array marking each word as valid or invalid
        validArray = [];
        for word in wordArray:
            validArray.append(reWordIsValid.match(word) != None)        
            
        # Tabulate total raw ngrams for this line into counts for each ngram bin
        # The total ngrams counts include the counts of all ngrams including those
        # that we won't consider as parts of phrases
        for j in range(1,5):
            if j<=numWords:
                ngramCounts[j] += numWords - j + 1 
        
        # Collect counts for viable phrase ngrams and left context sub-phrases
        for j in range(0,numWords):
            word = wordArray[j]

            # Only bother counting the ngrams that start with a valid content word
            # i.e., valids words not in the function word list or the black list
            if ( ( word not in functionwordHash ) and ( word not in blacklistHash ) and validArray[j] ):

                # Initialize ngram string with first content word and add it to unigram counts
                ngramSeq = word 
                if ngramSeq in ngramStats[1]:
                    ngramStats[1][ngramSeq] += 1
                else:
                    ngramStats[1][ngramSeq] = 1

                # Count valid ngrams from bigrams up to 4-grams
                stop = 0
                k = 1
                while (k<4) and (j+k<numWords) and not stop:
                    n = k + 1
                    nextNgramWord = wordArray[j+k]
                    # Only count ngrams with valid words not in the blacklist
                    if ( validArray[j+k] and nextNgramWord not in blacklistHash ):
                        ngramSeq += " " + nextNgramWord
                        if ngramSeq in ngramStats[n]:
                            ngramStats[n][ngramSeq] += 1
                        else:
                            ngramStats[n][ngramSeq] = 1 
                        k += 1
                        if nextNgramWord not in functionwordHash:
                            # Stop counting new ngrams after second content word in 
                            # ngram is reached and ngram is a viable full phrase
                            stop = 1
                    else:
                        stop = 1
    return ngramStats

### Rank Potential Phrases by the Weighted Pointwise Mutual Information of their Constituent Words

In [10]:
def RankNgrams(ngramStats,functionwordHash,minCount):
    # Create a hash table to store weighted pointwise mutual 
    # information scores for each viable phrase
    ngramWPMIHash = {}
        
    # Go through each of the ngram tables and compute the phrase scores
    # for the viable phrases
    for n in range(2,5):
        i = n-1
        for ngram in ngramStats[n].keys():
            ngramCount = ngramStats[n][ngram]
            if ngramCount >= minCount:
                wordArray = ngram.split()
                # If the final word in the ngram is not a function word then
                # the ngram is a valid phrase candidate we want to score
                if wordArray[i] not in functionwordHash: 
                    leftNgram = wordArray[0]
                    for j in range(1,i):
                        leftNgram += ' ' + wordArray[j]
                    rightWord = wordArray[i]
                    
                    # Compute the weighted pointwise mutual information (WPMI) for the phrase
                    probNgram = float(ngramStats[n][ngram])/float(ngramStats[0][n])
                    probLeftNgram = float(ngramStats[n-1][leftNgram])/float(ngramStats[0][n-1])
                    probRightWord = float(ngramStats[1][rightWord])/float(ngramStats[0][1])
                    WPMI = probNgram * math.log(probNgram/(probLeftNgram*probRightWord));

                    # Add the phrase into the list of scored phrases only if WMPI is positive
                    if WPMI > 0:
                        ngramWPMIHash[ngram] = WPMI  
    
    # Create a sorted list of the phrase candidates
    rankedNgrams = sorted(ngramWPMIHash, key=ngramWPMIHash.__getitem__, reverse=True)

    # Force a memory clean-up
    ngramWPMIHash = None
    gc.collect()

    return rankedNgrams

### Apply Phrase Rewrites to Train Data

In [11]:
def ApplyPhraseRewrites(rankedNgrams,textData,learnedPhrases,                 
                        maxPhrasesToAdd,maxPhraseLength,verbose):
    
    if len(rankedNgrams) == 0:
        return
    
    # This function will consider at most maxRewrite 
    # new phrases to be added into the learned phrase 
    # list as specified by the calling fuinction
    maxRewrite=maxPhrasesToAdd

    # If the remaining number of proposed ngram phrases is less 
    # than the max allowed, then reset maxRewrite to the size of 
    # the proposed ngram phrases list
    numNgrams = len(rankedNgrams)
    if numNgrams < maxRewrite:
        maxRewrite = numNgrams
    
    # Create empty hash tables to keep track of phrase overlap conflicts
    leftConflictHash = {}
    rightConflictHash = {}
    
    # Create an empty hash table collecting the set of rewrite rules
    # to be applied during this iteration of phrase learning
    ngramRewriteHash = {}
    
    # Precompile the regex for finding spaces in ngram phrases
    regexSpace = re.compile(' ')

    # Initialize some bookkeeping variables
    numLines = len(textData)
    numPhrasesAdded = 0
    numConsidered = 0
    lastSkippedNgram = ""
    lastAddedNgram = ""
  
    # Collect list up to maxRewrite ngram phrase rewrites
    stop = False
    index = 0
    while not stop:

        # Get the next phrase to consider adding to the phrase list
        inputNgram = rankedNgrams[index]

        # Create the output compound word version of the phrase
        # The extra space is added to make the regex rewrite easier
        outputNgram = " " + regexSpace.sub("_",inputNgram)

        # Count the total number of words in the proposed phrase
        numWords = len(outputNgram.split("_"))

        # Only add phrases that don't exceed the max phrase length
        if (numWords <= maxPhraseLength):
    
            # Keep count of phrases considered for inclusion during this iteration
            numConsidered += 1

            # Extract the left and right words in the phrase to use
            # in checks for phrase overlap conflicts
            ngramArray = inputNgram.split()
            leftWord = ngramArray[0]
            rightWord = ngramArray[len(ngramArray)-1]

            # Skip any ngram phrases that conflict with earlier phrases added
            # These ngram phrases will be reconsidered in the next iteration
            if (leftWord in leftConflictHash) or (rightWord in rightConflictHash): 
                if verbose: 
                    print ("(%d) Skipping (context conflict): %s" % (numConsidered,inputNgram))
                lastSkippedNgram = inputNgram
                
            # If no conflict exists then add this phrase into the list of phrase rewrites     
            else: 
                if verbose:
                    print ("(%d) Adding: %s" % (numConsidered,inputNgram))
                ngramRewriteHash[" " + inputNgram] = outputNgram
                learnedPhrases.append(inputNgram) 
                lastAddedNgram = inputNgram
                numPhrasesAdded += 1
            
            # Keep track of all context words that might conflict with upcoming
            # propose phrases (even when phrases are skipped instead of added)
            leftConflictHash[rightWord] = 1
            rightConflictHash[leftWord] = 1

            # Stop when we've considered the maximum number of phrases per iteration
            if ( numConsidered >= maxRewrite ):
                stop = True
            
        # Increment to next phrase
        index += 1
    
        # Stop if we've reached the end of the ranked ngram list
        if index >= len(rankedNgrams):
            stop = True

    # Now do the phrase rewrites over the entire set of text data
    if numPhrasesAdded == 1:
        # If only one phrase to add use a single regex rule to do this phrase rewrite        
        inputNgram = " " + lastAddedNgram
        outputNgram = ngramRewriteHash[inputNgram]
        regexNgram = re.compile (r'%s(?= )' % re.escape(inputNgram)) 
        # Apply the regex over the full data set
        for j in range(0,numLines):
            textData[j] = regexNgram.sub(outputNgram, textData[j])
    elif numPhrasesAdded > 1:
        # Compile a single regex rule from the collected set of phrase rewrites for this iteration
        ngramRegex = re.compile(r'%s(?= )' % "|".join(map(re.escape, ngramRewriteHash.keys())))
        # Apply the regex over the full data set
        for i in range(0,len(textData)):
            # The regex substituion looks up the output string rewrite  
            # in the hash table for each matched input phrase regex
            textData[i] = ngramRegex.sub(lambda mo: ngramRewriteHash[mo.string[mo.start():mo.end()]], textData[i]) 
      
    return

### Run the full iterative phrase learning process

In [12]:
def ApplyPhraseLearning(textData,learnedPhrases,learningSettings):
    
    stop = 0
    iterNum = 0

    # Get the learning parameters from the structue passed in by thee calling function
    maxNumPhrases = learningSettings.maxNumPhrases
    maxPhraseLength = learningSettings.maxPhraseLength
    functionwordHash = learningSettings.functionwordHash
    blacklistHash = learningSettings.blacklistHash
    verbose = learningSettings.verbose
    minCount = learningSettings.minInstanceCount
    
    # Start timing the process
    functionStartTime = time.clock()
    
    numPhrasesLearned = len(learnedPhrases)
    print ("Start phrase learning with %d phrases of %d phrases learned" % (numPhrasesLearned,maxNumPhrases))

    while not stop:
        iterNum += 1
                
        # Start timing this iteration
        startTime = time.clock()
 
        # Collect ngram stats
        ngramStats = ComputeNgramStats(textData,functionwordHash,blacklistHash)

        # Rank ngrams
        rankedNgrams = RankNgrams(ngramStats,functionwordHash,minCount)
        
        # Incorporate top ranked phrases into phrase list
        # and rewrite the text to use these phrases
        maxPhrasesToAdd = maxNumPhrases - numPhrasesLearned
        if maxPhrasesToAdd > learningSettings.maxPhrasesPerIter:
            maxPhrasesToAdd = learningSettings.maxPhrasesPerIter
        ApplyPhraseRewrites(rankedNgrams,textData,learnedPhrases,maxPhrasesToAdd,maxPhraseLength,verbose)
        numPhrasesAdded = len(learnedPhrases) - numPhrasesLearned

        # Garbage collect
        ngramStats = None
        rankedNgrams = None
        gc.collect();
               
        elapsedTime = time.clock() - startTime

        numPhrasesLearned = len(learnedPhrases)
        print ("Iteration %d: Added %d new phrases in %.2f seconds (Learned %d of max %d)" % 
               (iterNum,numPhrasesAdded,elapsedTime,numPhrasesLearned,maxNumPhrases))
        
        if numPhrasesAdded >= maxPhrasesToAdd or numPhrasesAdded == 0:
            stop = 1
        
    # Remove the space padding at the start and end of each line
    regexSpacePadding = re.compile('^ +| +$')
    for i in range(0,len(textData)):
        textData[i] = regexSpacePadding.sub("",textData[i])
    
    gc.collect()
 
    elapsedTime = time.clock() - functionStartTime
    elapsedTimeHours = elapsedTime/3600.0;
    print ("*** Phrase learning completed in %.2f hours ***" % elapsedTimeHours) 

    return

In [13]:
# Create a structure defining the settings and word lists used during the phrase learning
learningSettings = namedtuple('learningSettings',['maxNumPhrases','maxPhrasesPerIter',
                                                  'maxPhraseLength','minInstanceCount'
                                                  'functionwordHash','blacklistHash','verbose'])

# If true it prints out the learned phrases to stdout buffer
# while its learning. This will generate a lot of text to stdout, 
# so best to turn this off except for testing and debugging
learningSettings.verbose = False

# Maximium number of phrases to learn
# If you want to test the code out quickly then set this to a small
# value (e.g. 100) and set verbose to true when running the quick test
learningSettings.maxNumPhrases = 200

# Maximum number of phrases to learn per iteration 
# Increasing this number may speed up processing but will affect the ordering of the phrases 
# learned and good phrases could be by-passed if the maxNumPhrases is set to a small number
learningSettings.maxPhrasesPerIter = 50

# Maximum number of words allowed in the learned phrases 
learningSettings.maxPhraseLength = 7

# Minimum number of times a phrase must occur in the data to 
# be considered during the phrase learning process
learningSettings.minInstanceCount = 5

# This is a precreated hash table containing the list 
# of function words used during phrase learning
learningSettings.functionwordHash = functionwordHash

# This is a precreated hash table containing the list 
# of black list words to be ignored during phrase learning
learningSettings.blacklistHash = {}

In [14]:
###### Questions:
# Initialize an empty list of learned phrases
# If you have completed a partial run of phrase learning
# and want to add more phrases, you can use the pre-learned 
# phrases as a starting point instead and the new phrases
# will be appended to the list
learnedPhrasesQ = []

# Create a copy of the original text data that will be used during learning
# The copy is needed because the algorithm does in-place replacement of learned
# phrases directly on the text data structure it is provided
phraseTextDataQ = []
for textLine in CleanedTrainQ['LowercaseText']:
    phraseTextDataQ.append(' ' + textLine + ' ')

# Run the phrase learning algorithm
ApplyPhraseLearning(phraseTextDataQ,learnedPhrasesQ,learningSettings)

# Add text with learned phrases back into data frame
CleanedTrainQ['TextWithPhrases'] = phraseTextDataQ

Start phrase learning with 0 phrases of 200 phrases learned
Iteration 1: Added 42 new phrases in 1.14 seconds (Learned 42 of max 200)
Iteration 2: Added 35 new phrases in 1.08 seconds (Learned 77 of max 200)
Iteration 3: Added 32 new phrases in 1.07 seconds (Learned 109 of max 200)
Iteration 4: Added 34 new phrases in 1.08 seconds (Learned 143 of max 200)
Iteration 5: Added 31 new phrases in 1.06 seconds (Learned 174 of max 200)
Iteration 6: Added 11 new phrases in 1.00 seconds (Learned 185 of max 200)
Iteration 7: Added 3 new phrases in 0.98 seconds (Learned 188 of max 200)
Iteration 8: Added 4 new phrases in 0.97 seconds (Learned 192 of max 200)
Iteration 9: Added 1 new phrases in 0.97 seconds (Learned 193 of max 200)
Iteration 10: Added 1 new phrases in 0.96 seconds (Learned 194 of max 200)
Iteration 11: Added 1 new phrases in 0.96 seconds (Learned 195 of max 200)
Iteration 12: Added 1 new phrases in 0.96 seconds (Learned 196 of max 200)
Iteration 13: Added 1 new phrases in 1.04 sec

### Apply the Learned Phrases to Test Data

In [15]:
def ApplyPhraseRewritesInPlace(textFrame, textColumnName, phraseRules):
        
    # Get text data column from frame
    textData = textFrame[textColumnName]
    numLines = len(textData)
    
    # initial a list to store output text
    textOutput = [None] * numLines
    
    # Add leading and trailing spaces to make regex matching easier
    for i in range(0,numLines):
        textOutput[i] = " " + textData[i] + " "  

    # Make sure we have phrase to add
    numPhraseRules = len(phraseRules)
    if numPhraseRules == 0: 
        print ("Warning: phrase rule lise is empty - no phrases being applied to text data")
        return

    # Precompile the regex for finding spaces in ngram phrases
    regexSpace = re.compile(' ')
   
    # Initialize some bookkeeping variables

    # Iterate through full set of phrases to find sets of 
    # non-conflicting phrases that can be apply simultaneously
    index = 0
    outerStop = False
    while not outerStop:
       
        # Create empty hash tables to keep track of phrase overlap conflicts
        leftConflictHash = {}
        rightConflictHash = {}
        prevConflictHash = {}
    
        # Create an empty hash table collecting the next set of rewrite rules
        # to be applied during this iteration of phrase rewriting
        phraseRewriteHash = {}
    
        # Progress through phrases until the next conflicting phrase is found
        innerStop = 0
        numPhrasesAdded = 0
        while not innerStop:
        
            # Get the next phrase to consider adding to the phrase list
            nextPhrase = phraseRules[index]            
            
            # Extract the left and right sides of the phrase to use
            # in checks for phrase overlap conflicts
            ngramArray = nextPhrase.split()
            leftWord = ngramArray[0]
            rightWord = ngramArray[len(ngramArray)-1] 

            # Stop if we reach any phrases that conflicts with earlier phrases in this iteration
            # These ngram phrases will be reconsidered in the next iteration
            if ((leftWord in leftConflictHash) or (rightWord in rightConflictHash) 
                or (leftWord in prevConflictHash) or (rightWord in prevConflictHash)): 
                innerStop = True
                
            # If no conflict exists then add this phrase into the list of phrase rewrites     
            else: 
                # Create the output compound word version of the phrase
                                
                outputPhrase = regexSpace.sub("_",nextPhrase);
                
                # Keep track of all context words that might conflict with upcoming
                # propose phrases (even when phrases are skipped instead of added)
                leftConflictHash[rightWord] = 1
                rightConflictHash[leftWord] = 1
                prevConflictHash[outputPhrase] = 1           
                
                # Add extra space to input an output versions of the current phrase 
                # to make the regex rewrite easier
                outputPhrase = " " + outputPhrase
                lastAddedPhrase = " " + nextPhrase
                
                # Add the phrase to the rewrite hash
                phraseRewriteHash[lastAddedPhrase] = outputPhrase
                  
                # Increment to next phrase
                index += 1
                numPhrasesAdded  += 1
    
                # Stop if we've reached the end of the phrases list
                if index >= numPhraseRules:
                    innerStop = True
                    outerStop = True
                    
        # Now do the phrase rewrites over the entire set of text data
        if numPhrasesAdded == 1:
        
            # If only one phrase to add use a single regex rule to do this phrase rewrite        
            outputPhrase = phraseRewriteHash[lastAddedPhrase]
            regexPhrase = re.compile (r'%s(?= )' % re.escape(lastAddedPhrase)) 
        
            # Apply the regex over the full data set
            for j in range(0,numLines):
                textOutput[j] = regexPhrase.sub(outputPhrase, textOutput[j])
       
        elif numPhrasesAdded > 1:
            # Compile a single regex rule from the collected set of phrase rewrites for this iteration
            regexPhrase = re.compile(r'%s(?= )' % "|".join(map(re.escape, phraseRewriteHash.keys())))
            
            # Apply the regex over the full data set
            for i in range(0,numLines):
                # The regex substituion looks up the output string rewrite  
                # in the hash table for each matched input phrase regex
                textOutput[i] = regexPhrase.sub(lambda mo: phraseRewriteHash[mo.string[mo.start():mo.end()]], textOutput[i]) 
    
    # Remove the space padding at the start and end of each line
    regexSpacePadding = re.compile('^ +| +$')
    for i in range(0,len(textOutput)):
        textOutput[i] = regexSpacePadding.sub("",textOutput[i])
    
    return textOutput

In [16]:
CleanedTestQ['TextWithPhrases'] = ApplyPhraseRewritesInPlace(CleanedTestQ, 'LowercaseText', learnedPhrasesQ)

In [17]:
# show an example of cleaned testQ
CleanedTestQ.loc[9]

DocID                              23830543
DocLine                                   9
CleanedText        it seems that this works
LowercaseText      it seems that this works
TextWithPhrases    it seems that this works
Name: 9, dtype: object

### Reconstruct the Full Processed Text of Each Document and Put it into a New Frame¶

In [18]:
def ReconstituteDocsFromChunks(textData, idColumnName, textColumnName):
    dataOut = []
    
    currentDoc = "";
    currentDocID = "";
    
    for i in range(0,len(textData)):
        textChunk = textData[textColumnName][i]
        docID = textData[idColumnName][i]
        if docID != currentDocID:
            if currentDocID != "":
                dataOut.append(currentDoc)
            currentDoc = textChunk
            currentDocID = docID
        else:
            currentDoc += " " + textChunk
    dataOut.append(currentDoc)
    
    return dataOut

In [19]:
trainQ['TextWithPhrases'] = ReconstituteDocsFromChunks(CleanedTrainQ, 'DocID', 'TextWithPhrases')
testQ['TextWithPhrases'] = ReconstituteDocsFromChunks(CleanedTestQ, 'DocID', 'TextWithPhrases')

### Create the Vocabulary With Filtering Criteria

In [20]:
def CreateVocabForTopicModeling(textData,stopwordHash):

    print ("Counting words")
    numDocs = len(textData) 
    globalWordCountHash = {} 
    globalDocCountHash = {} 
    for textLine in textData:
        docWordCountHash = {}
        for word in textLine.split():
            if word in globalWordCountHash:
                globalWordCountHash[word] += 1
            else:
                globalWordCountHash[word] = 1
            if word not in docWordCountHash: 
                docWordCountHash[word] = 1
                if word in globalDocCountHash:
                    globalDocCountHash[word] += 1
                else:
                    globalDocCountHash[word] = 1

    minWordCount = 5;
    minDocCount = 2;
    maxDocFreq = .25;
    vocabCount = 0;
    vocabHash = {}

    excStopword = 0
    excNonalphabetic = 0
    excMinwordcount = 0
    excNotindochash = 0
    excMindoccount = 0
    excMaxdocfreq =0

    print ("Building vocab")
    for word in globalWordCountHash.keys():
        # Test vocabulary exclusion criteria for each word
        if ( word in stopwordHash ):
            excStopword += 1
        elif ( not re.search(r'[a-zA-Z]', word, 0) ):
            excNonalphabetic += 1
        elif ( globalWordCountHash[word] < minWordCount ):
            excMinwordcount += 1
        elif ( word not in globalDocCountHash ):
            print ("Warning: Word '%s' not in doc count hash") % (word)
            excNotindochash += 1
        elif ( globalDocCountHash[word] < minDocCount ):
            excMindoccount += 1
        elif ( float(globalDocCountHash[word])/float(numDocs) > maxDocFreq ):
            excMaxdocfreq += 1
        else:
            # Add word to vocab
            vocabHash[word]= globalWordCountHash[word];
            vocabCount += 1 
    print ("Excluded %d stop words" % (excStopword))       
    print ("Excluded %d non-alphabetic words" % (excNonalphabetic))  
    print ("Excluded %d words below word count threshold" % (excMinwordcount)) 
    print ("Excluded %d words below doc count threshold" % (excMindoccount))
    print ("Excluded %d words above max doc frequency" % (excMaxdocfreq)) 
    print ("Final Vocab Size: %d words" % vocabCount)
            
    return vocabHash

In [21]:
vocabHashQ = CreateVocabForTopicModeling(trainQ['TextWithPhrases'],functionwordHash)

Counting words
Building vocab
Excluded 307 stop words
Excluded 911 non-alphabetic words
Excluded 15276 words below word count threshold
Excluded 142 words below doc count threshold
Excluded 3 words above max doc frequency
Final Vocab Size: 3116 words


### Tokenize Text with Learned Phrases

In [22]:
# start by tokenizing the full text string string for each document into list of tokens.
# any token that is in not in the pre-defined set of acceptable vocabulary words is execluded.
def TokenizeText(textData,vocabHash):
    tokenizedText = ''
    for token in textData.split():
        if token in vocabHash:
            tokenizedText += (token.strip() + ',')
    return tokenizedText.strip(',')

In [23]:
trainQ['Tokens'] = trainQ['TextWithPhrases'].apply(lambda x: TokenizeText(x, vocabHashQ))
testQ['Tokens'] = testQ['TextWithPhrases'].apply(lambda x: TokenizeText(x, vocabHashQ))

In [24]:
# an example of tokenized text in training set.
trainQ['Tokens'].iloc[0]

'self-closing,script,tags,work,reason,browsers,correctly,recognize,recognized,break,concept,xhtml,support,note,statement,correct,all,ie'

## Feature Extraction

Selecting the right set of features is very critical for the model training. Therefore, we're going to show you several feature extraction approaches that have been testied to perform well in the text classification use cases.

In [25]:
# get Token to ID mapping: {Token: tokenId}
def tokensToIds(tokens, featureHash):
    token2IdHash = {}
    for i in range(len(tokens)):
        tokenList = tokens.iloc[i].split(',')
        if featureHash is None:
            for t in tokenList:
                if t not in token2IdHash.keys():
                    token2IdHash[t] = len(token2IdHash)
        else:
            for t in tokenList:
                if t not in token2IdHash.keys() and t in list(featureHash.keys()):
                    token2IdHash[t] = len(token2IdHash)
            
    return token2IdHash

##########################################

def countMatrix(frame, token2IdHash, labelColumnName=None, uniqueLabel=None):
    # create am empty matrix with the shape of:
    # num_row = num of unique tokens
    # num_column = num of unique answerIds (N_wA) or num of questions in testQ (tfMatrix)
    # rowIdx = token2IdHash.values()
    # colIdx = index of uniqueClass (N_wA) or index of questions in testQ (tfMatrix)
    num_row = len(token2IdHash)
    if uniqueLabel is not None:  # get N_wA
        num_column = len(uniqueLabel)
    else:
        num_column = len(frame)
    countMatrix = np.zeros(shape=(num_row, num_column))

    # loop through each question in the frame to fill in the countMatrix with corresponding counts
    for i in range(len(frame)):
        tokens = frame['Tokens'].iloc[i].split(',')
        if uniqueLabel is not None:   # get N_wA
            label = frame[labelColumnName].iloc[i]
            colIdx = uniqueLabel.index(label)
        else:
            colIdx = i
            
        for t in tokens:
            if t in token2IdHash.keys():
                rowIdx = token2IdHash[t]
                countMatrix[rowIdx, colIdx] += 1

    return countMatrix

##############################################

# calculate P(A): [P_A1, P_A2, ...]
def priorProbabilityAnswer(answerIds, uniqueLabel): 
    P_A = []
    # convert a pandas series to a list
    answerIds = list(answerIds)
    
    for id in uniqueLabel:
        P_A.append(answerIds.count(id)/len(answerIds))
    return np.array(P_A)

##############################################

# calculate P(A|w)
def posterioriProb(N_wAInit, P_A, uniqueLabel):
    # N_A is the total number of answers
    N_A = len(uniqueLabel)
    # N_w is the total number of times w appears over all documents 
    # rowSum of count matrix (N_wAInit)
    N_wInit = np.sum(N_wAInit, axis = 1)
    # P(A|w) = (N_w|A + N_A * P(A))/(N_w + N_A)
    N = N_wAInit + N_A * P_A
    D = N_wInit + N_A
    P_Aw = np.divide(N.T, D).T    
    
    return P_Aw

##############################################

# select the top N tokens w which maximize P(A|w) for each A.
# get FeatureHash: {token: 1}
def feature_selection(P_Aw, token2IdHashInit, topN):
    featureHash = {}
    # for each answer A, sort tokens w by P(A|w)
    sortedIdxMatrix = np.argsort(P_Aw, axis=0)[::-1]
    # select top N tokens for each answer A
    topMatrix = sortedIdxMatrix[0:topN, :]
    # for each token w in topMatrix, add w to FeatureHash if it has not already been included
    topTokenIdList = np.reshape(topMatrix, topMatrix.shape[0] * topMatrix.shape[1])
    # get ID to Token mapping: {tokenId: Token}
    Id2TokenHashInit = {y:x for x, y in token2IdHashInit.items()}
    
    for tokenId in topTokenIdList:
        token = Id2TokenHashInit[tokenId]
        if token not in featureHash.keys():
            featureHash[token] = 1
    return featureHash

##############################################

def featureWeights(N_wA, alpha):
    # N_w is the total number of times w appears over all documents 
    # rowSum of count matrix (N_wA)
    N_w = np.sum(N_wA, axis = 1)
    # N_W is the total count of all words
    N_W = np.sum(N_wA)
    # N_V is the count of unique words in the vocabulary
    N_V = N_wA.shape[0]
    # P(w) = (N_w + 1*alpha) / (N_W +N_V*alpha)
    N2 = N_w + 1 * alpha
    D2 = N_W + alpha * N_V
    P_w = N2/D2

    return P_w

##############################################

def wordProbabilityInAnswer(N_wA, P_w, beta):
    # N_V is the count of unique words in the vocabulary
    N_V = N_wA.shape[0]
    # N_WA is the total count of all words in questions on answer A 
    # colSum of count matrix (N_wA)
    N_WA = np.sum(N_wA, axis=0)
    # P(w|A) = (N_w|A + beta N_V P(w))/(N_W|A + beta * N_V)
    N = (N_wA.T + beta * N_V * P_w).T
    D = N_WA + beta * N_V
    P_wA = N / D
    
    return P_wA

################################################

def wordProbabilityNotinAnswer(N_wA, P_w, beta):
    # N_V is the count of unique words in the vocabulary
    N_V = N_wA.shape[0]
    # N_wNotA is the count of w over all documents but not on answer A
    # N_wNotA = N_w - N_wA
    N_w = np.sum(N_wA, axis = 1)
    N_wNotA = (N_w - N_wA.T).T
    # N_WNotA is the count of all words over all documents but not on answer A
    # N_WNotA = N_W - N_WA
    N_W = np.sum(N_wA)
    N_WA = np.sum(N_wA, axis=0)
    N_WNotA = N_W - N_WA
    # P(w|NotA) = (N_w|NotA + beta * N_V * P(w))/(N_W|NotA + beta * N_V)
    N = (N_wNotA.T + beta * N_V * P_w).T
    D = N_WNotA + beta * N_V
    P_wNotA = N / D
    
    return P_wNotA

#################################################

def normalizeTF(frame, token2IdHash):
    
    N_wQ = countMatrix(frame, token2IdHash)
    N_WQ = np.sum(N_wQ, axis=0)
    
    # find the index where N_WQ is zero
    zeroIdx = np.where(N_WQ == 0)[0]
    
    # if N_WQ is zero, then the x_w for that particular question would be zero.
    # for a simple calculation, we convert the N_WQ to 1 in those cases so the demoninator is not zero. 
    if len(zeroIdx) > 0:
        N_WQ[zeroIdx] = 1
    
    # x_w = P_wd = count(w)/sum(count(i in V))
    x_w = N_wQ / N_WQ
    
    return x_w

#####################################################

def getIDF(N_wQ):
    # N is total number of documents in the corpus
    # N_V is the number of tokens in the vocabulary
    N_V, N = N_wQ.shape
    # D is the number of documents where the token w appears
    D = np.zeros(shape=(0, N_V))
    for i in range(N_V):
        D = np.append(D, len(np.nonzero(N_wQ[i, ])[0]))
    return np.log(N/D)

#####################################################

def softmax(scores2D):
    # input: scores from different models
    # row: test example
    # column: label
    return np.exp(scores2D)/np.sum(np.exp(scores2D), axis=1)[:, None]

#####################################################

# train one-vs-rest classifier using NB scores as features.
def ovrClassifier(trainLabels, x_wTrain, x_wTest, NBWeights, clf, ratio):
    uniqueLabel = np.unique(trainLabels)
    dummyLabels = pd.get_dummies(trainLabels)
    numTest = x_wTest.shape[1]
    Y_test_prob = np.zeros(shape=(numTest, len(uniqueLabel)))

    for i in range(len(uniqueLabel)):
        X_train_all, Y_train_all = x_wTrain.T * NBWeights[:, i], dummyLabels.iloc[:, i]
        X_test = x_wTest.T * NBWeights[:, i]
        
        # with sample selection.
        if ratio is not None:
            # ratio = # of Negative/# of Positive
            posIdx = np.where(Y_train_all == 1)[0]
            negIdx = np.random.choice(np.where(Y_train_all == 0)[0], ratio*len(posIdx))
            allIdx = np.concatenate([posIdx, negIdx])
            X_train, Y_train = X_train_all[allIdx], Y_train_all.iloc[allIdx]
        else: # without sample selection.
            X_train, Y_train = X_train_all, Y_train_all
            
        clf.fit(X_train, Y_train)
        if hasattr(clf, "decision_function"):
            Y_test_prob[:, i] = clf.decision_function(X_test)
        else:
            Y_test_prob[:, i] = clf.predict_proba(X_test)[:, 1]

    return softmax(Y_test_prob)

### Term Frequency and Inverse Document Frequency (TF-IDF) 

TF-IDF is commonly used as features when training text classification models. 

Each document d is typically represented by a feature vector x that represents the contents of d. Because different documents can have different lengths, it can be useful to apply L1 normalized feature vector x. Therefore, a normalized __Term Frequency__ matrix can be obtained based on the below formula.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/tf.PNG?token=APoO9tMyEVzqoUJYT9ALcdF3_BryHHEVks5YnIQywA%3D%3D">

Considering all tokens observed in the training questions and answers, we compute their __Inverse Document Frequency__ based on the below formula.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/idf.PNG?token=APoO9qf3WptQgUPVRQJuOt4cobf56-Y3ks5YnhLSwA%3D%3D">

By knowing the __Term Frequency (TF)__ matrix and __Inverse Document Frequency (IDF)__ vector, we can simply compute __TF-IDF__ matrix by multiplying them together.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/tfidf.PNG?token=APoO9gw3rPhLusbG3if65TuVZNAnyqTCks5YnhWPwA%3D%3D">

In [26]:
token2IdHashInit = tokensToIds(trainQ['Tokens'], featureHash=None)
# get unique answerId in ascending order
uniqueAnswerId = list(np.unique(trainQ['AnswerId']))


###### TF-IDF #######
N_wQ = countMatrix(trainQ, token2IdHashInit)
idf = getIDF(N_wQ)

x_wTest = normalizeTF(testQ, token2IdHashInit)
x_wTrain = normalizeTF(trainQ, token2IdHashInit)

tfidfTest = (x_wTest.T * idf).T
tfidfTrain = (x_wTrain.T * idf).T

### Naive Bayes Scores

Besides using the IDF as the word weighting mechnism, a hypothesis testing likelihood ratio approach is also implemented here. 

In this approach, the word weights are associated with the answer classes and are calculated using the below formula.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/NB_weight.PNG?token=APoO9s-2DejCvW03RgK6zXgiXX6UT5WWks5YnjiRwA%3D%3D">

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/probability_function.PNG?token=APoO9rWEZ1g_OgvWT_pleQlhT2DEFw3tks5YnIHzwA%3D%3D">

By knowing the __Term Frequency (TF)__ matrix and __Weight__ vector for each class, we can simply compute __Naive Bayes Scores__ matrix for each class by multiplying them together.

In [27]:
###### Naive Bayes Scores #####
# calculate the count matrix of all training questions.
N_wAInit = countMatrix(trainQ, token2IdHashInit, 'AnswerId', uniqueAnswerId)

P_A = priorProbabilityAnswer(trainQ['AnswerId'], uniqueAnswerId)
P_Aw = posterioriProb(N_wAInit, P_A, uniqueAnswerId)

featureHash = feature_selection(P_Aw, token2IdHashInit, topN=20)
token2IdHash = tokensToIds(trainQ['Tokens'], featureHash=featureHash)

N_wA = countMatrix(trainQ, token2IdHash, 'AnswerId', uniqueAnswerId)

alpha = 0.0001
P_w = featureWeights(N_wA, alpha)

beta = 0.0001
P_wA = wordProbabilityInAnswer(N_wA, P_w, beta)
P_wNotA = wordProbabilityNotinAnswer(N_wA, P_w, beta)

NBWeights = np.log(P_wA / P_wNotA)

x_wTrain = normalizeTF(trainQ, token2IdHash)
x_wTest = normalizeTF(testQ, token2IdHash)

## Model Training

### Naive Bayes Classifier

We implement the _Naive Bayes Classifier_ as described in the paper entitled ["MCE Training Techniques for Topic Identification of Spoken Audio Documents"](http://ieeexplore.ieee.org/abstract/document/5742980/).

In [28]:
beta_A = 0
Y_test_prob1 = softmax(-beta_A + np.dot(x_wTest.T, NBWeights))

### Support Vector Machine (TF-IDF as features)

Traditional SVM training finds a hyperplane which maximally seperates positive and negative training tokens in a vector space. In its standard form, an SVM is a two-class classifier. To create a multi-class SVM for a problem with N_A classes, a one-versus-rest SVM classifier is typically learned for each answer class a.

In [29]:
X_train, Y_train = tfidfTrain.T, np.array(trainQ['AnswerId'])
X_test = tfidfTest.T
clf = svm.LinearSVC(dual=True, multi_class='ovr', penalty='l2', C=1, loss="squared_hinge", random_state=1)
clf.fit(X_train, Y_train)
%time Y_test_prob2 = softmax(clf.decision_function(X_test))

### Random Forest (NB Scores as features)

Similar to the above one-versus-rest SVM classifier, we also implement a one-versus-rest Random Forest classifier. 

In each two-class base classifier, we dynamically compute the naive bayes scores for the positive class as the features. Since the number of negative examples is much larger than the number of positive examples, we hold all positive example and randomly select negative examples based on a negative to positive ratio to obtain a balanced training data. 

In [30]:
clf = RandomForestClassifier(n_estimators=250, criterion='entropy', random_state=1)
%time Y_test_prob3 = ovrClassifier(trainQ["AnswerId"], x_wTrain, x_wTest, NBWeights, clf, ratio=3)

Wall time: 1min 2s


### Ensemble Model

We build an ensemble model by combining the predicted probabilities from three previously trained classifiers. 

In [31]:
Y_test_prob_aggr = np.mean([Y_test_prob1, Y_test_prob2, Y_test_prob3], axis=0)

## Evaluate Model Performance

We use two evaluation matrices to test our model performance. For each question in the test set, we calculate a weighted average of the probabilities obtained from the base classifiers against each answer. Then we rank the answers based on their weighted average to calculate __Average Rank__ and __Top 10 Percentage__ in the Test set using the below formula:

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/evaluation.PNG?token=APoO9hyYDFxGc9FRbmIXU3VGv0wdeCaPks5YnIVtwA%3D%3D">

The __Average Rank__ can be interpreted as in average at which position we can find the correct answer among all available answers for a given question. 

The __Top 10 Percentage__ can be interpreted as how many percentage of the new questions that we can find their correct answers in the first 10 choices.

In [32]:
# sort the similarity scores in descending order and map them to the corresponding AnswerId in Answer set
def rank(frame, scores, uniqueAnswerId):
    frame['SortedAnswers'] = list(np.array(uniqueAnswerId)[np.argsort(-scores, axis=1)])
    
    rankList = []
    for i in range(len(frame)):
        rankList.append(np.where(frame['SortedAnswers'].iloc[i] == frame['AnswerId'].iloc[i])[0][0] + 1)
    frame['Rank'] = rankList
    
    return frame

In [33]:
testQ = rank(testQ, Y_test_prob_aggr, uniqueAnswerId)

In [37]:
print('Total number of questions in test set: ' + str(len(testQ)))
print('Total number of answers: ' + str(len(uniqueAnswerId)))
print('Total number of unique features: ' + str(len(featureHash)))
print('Average of rank: ' + str(np.floor(testQ['Rank'].mean())))
print('Percentage of questions find answers in top 3: ' + str(round(len(testQ.query('Rank <= 3'))/len(testQ), 3)))

Total number of questions in test set: 1735
Total number of answers: 103
Total number of unique features: 1295
Average of rank: 5.0
Percentage of questions find answers in top 3: 0.688
